In [2]:
!pip install pandas
import pandas as pd

In [3]:
df1 = pd.read_csv('house-prices-advanced-regression-techniques/train.csv', encoding='utf-8')
df2 = pd.read_csv('house-prices-advanced-regression-techniques/test.csv', encoding='utf-8')
df = pd.concat([df1, df2], axis=0, ignore_index=True)
print("dimension(row, col):", df.shape)

print("preview")
print(df.head())

print("data type and non-null value:")
print(df.info())

missing = df.isnull().sum().sort_values(ascending=False)
print("null value:")
print(missing[missing > 0])

print("stastical features:")
print(df.describe())

dimension(row, col): (2919, 81)
preview
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCond

In [4]:
import numpy as np

# Step 2: Missing Value Handling

missing_ratio = df.isnull().mean().sort_values(ascending=False)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

categorical_cols = df.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

# Drop columns with very high missing rate (>50%)
high_missing_cols = missing_ratio[missing_ratio > 0.5].index
df.drop(columns=high_missing_cols, inplace=True)

print("Missing values after imputation/dropping:")
print(df.isnull().sum()[df.isnull().sum() > 0])
print("dimension(row, col):", df.shape)


Missing values after imputation/dropping:
Series([], dtype: int64)
dimension(row, col): (2919, 76)


In [5]:
ordinal_levels = {'Po', 'Fa', 'TA', 'Gd', 'Ex'}

#Scan all object-dtype columns and pick those whose unique values ⊆ ordinal_levels
ordinal_candidates = []
for col in df.select_dtypes(include=['object']).columns:
    vals = set(df[col].dropna().unique())
    if vals.issubset(ordinal_levels):
        ordinal_candidates.append(col)

print("Columns with values only in", ordinal_levels, ":\n", ordinal_candidates)



Columns with values only in {'Fa', 'Gd', 'Ex', 'Po', 'TA'} :
 ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']


In [6]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder

# Step 4: Feature Encoding

ordinal_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
}

for col in ordinal_candidates:
    df[col] = df[col].map(ordinal_mapping)

remaining_cats = [
    c for c in df.select_dtypes(include=['object']).columns
    if c not in ordinal_candidates
]
df = pd.get_dummies(df, columns=remaining_cats, drop_first=True)

print("DataFrame shape after mapping:", df.shape)


#Simulate buyer preferences and encode
n_samples = df.shape[0]
n_buyers = 100  # arbitrary number of unique buyers
np.random.seed(42)
df['BuyerID'] = np.random.choice(np.arange(n_buyers), size=n_samples)
df['SizePreference'] = np.random.choice(['Small','Large'], size=n_samples)
df['NewOldPreference'] = np.random.choice(['Old','New'], size=n_samples)

# Encode BuyerID as integer labels (for random‐effects design matrix)
le = LabelEncoder()
df['BuyerID_enc'] = le.fit_transform(df['BuyerID'])

# Binary encode size and new/old preferences
df['SizePref_enc']    = df['SizePreference'].map({'Small':0, 'Large':1})
df['NewOldPref_enc']  = df['NewOldPreference'].map({'Old':0,   'New':1})

# (Optionally) drop the original categorical columns
df.drop(columns=['BuyerID','SizePreference','NewOldPreference'], inplace=True)

print("After encoding, dataframe shape:", df.shape)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 19.0 MB/s eta 0:00:00 0:00:01
DataFrame shape after mapping: (2919, 212)
After encoding, dataframe shape: (2919, 215)


In [7]:

from sklearn.preprocessing import StandardScaler

# Step 5: Feature Scaling

numeric_cols = df.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print(df[numeric_cols].head())



         Id  MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  \
0 -1.731458    0.067331    -0.191815 -0.217879     0.646183    -0.507284   
1 -1.730271   -0.873616     0.511940 -0.072044    -0.063185     2.188279   
2 -1.729084    0.067331    -0.051064  0.137197     0.646183    -0.507284   
3 -1.727897    0.302568    -0.426400 -0.078385     0.646183    -0.507284   
4 -1.726711    0.067331     0.699608  0.518903     1.355551    -0.507284   

   YearBuilt  YearRemodAdd  MasVnrArea  ExterQual  ...  3SsnPorch  \
0   1.046258      0.896833    0.529034   1.039805  ...  -0.103331   
1   0.154764     -0.395604   -0.567016  -0.683756  ...  -0.103331   
2   0.980221      0.848965    0.338903   1.039805  ...  -0.103331   
3  -1.859351     -0.682812   -0.567016  -0.683756  ...  -0.103331   
4   0.947203      0.753229    1.390216   1.039805  ...  -0.103331   

   ScreenPorch  PoolArea   MiscVal    MoSold    YrSold  SalePrice  \
0    -0.285935  -0.06315 -0.089592 -1.552184  0.157646   0.

In [8]:
# Seperate train dataset and test dataset
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['BuyerID_enc'], random_state=42)


In [9]:
# Save train_df and test_df to CSV files
train_df.to_csv('train_df_processed.csv', index=False)
test_df.to_csv('test_df_processed.csv', index=False)